In [33]:
import pyaudio
import wave
import os
import scipy.signal as signal
import librosa
import numpy as np
import glob

In [35]:
def englishPeaks():
    filepath = '/home/russell/SingIT/EnglishPeaks30.txt'
    EnglishF = [[] for i in range(90)]
    with open(filepath) as fp:
        line = fp.readline()
        cnt = 0
        while line:
            English = [0]*13
            line = fp.readline()
            values=line.split()
            for i in range(0,len(values)):
                    English[i]+=int(values[i])
            EnglishF[cnt]=English
            cnt+=1
    return EnglishF


In [36]:
def frenchPeaks():
    filepath2 = '/home/russell/SingIT/FrenchPeaks30.txt'
    FrenchF = [[] for i in range(90)]
    with open(filepath2) as fp:
        line = fp.readline()
        cnt = 0
        while line:
            French =  [0]*13
            line = fp.readline()
            values=line.split()
            for i in range(0,len(values)):
                    French[i]+=int(values[i])
            FrenchF[cnt]=French        
            cnt+=1           
    return FrenchF

In [37]:
def germanPeaks():
    GermanF = [[] for i in range(90)]
    filepath3 = '/home/russell/SingIT/GermanPeaks30.txt'
    with open(filepath3) as fp:
        line = fp.readline()
        cnt = 0
        while line:
            German =  [0]*13
            line = fp.readline()
            values=line.split()
            for i in range(0,len(values)):
                    German[i]+=int(values[i])
            GermanF[cnt]=German        
            cnt+=1
    return GermanF

In [38]:
def japanesePeaks():
    JapaneseF = [[] for i in range(90)]
    filepath4 = '/home/russell/SingIT/JapanesePeaks30.txt'
    with open(filepath4) as fp:
        line = fp.readline()
        cnt = 0
        while line:
            Japanese = [0]*13
            line = fp.readline()
            values=line.split()
            for i in range(0,len(values)):
                    Japanese[i]+=int(values[i])
            JapaneseF[cnt] = Japanese
            cnt+=1
    return JapaneseF

In [28]:
def betterVocalExtract(song):
    #song = '"'+song+'"'
    #command = "spleeter separate -i "+ song + " -p spleeter:2stems -o output"
    print("Start Command")
    !spleeter separate -i output.wav -p spleeter:2stems -o output
    print("Finished Command")

In [29]:
def readInLiveAudio():
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    RECORD_SECONDS = 10
    WAVE_OUTPUT_FILENAME = "output.wav"
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)
    print("* recording")
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    print("* done recording")
    stream.stop_stream()
    stream.close()
    p.terminate()
    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [30]:
def bandComparison(song):
    n = 10
    filtertrain = [[0.0]]*13
    filtertrain[0] = signal.butter(n, [3000,4000], 'bandpass', fs=44100, output='sos')
    filtertrain[1] = signal.butter(n, [4000,5000], 'bandpass', fs=44100, output='sos')
    filtertrain[2] = signal.butter(n, [5000,6000], 'bandpass', fs=44100, output='sos')
    filtertrain[3] = signal.butter(n, [6000,7000], 'bandpass', fs=44100, output='sos')
    filtertrain[4] = signal.butter(n, [7000,8000], 'bandpass', fs=44100, output='sos')
    filtertrain[5] = signal.butter(n, [8000,9000], 'bandpass', fs=44100, output='sos')
    filtertrain[6] = signal.butter(n, [9000,10000], 'bandpass', fs=44100, output='sos')
    filtertrain[7] = signal.butter(n, [10000,11000], 'bandpass', fs=44100, output='sos')
    filtertrain[8] = signal.butter(n, [11000,12000], 'bandpass', fs=44100, output='sos')
    filtertrain[9] = signal.butter(n, [12000,13000], 'bandpass', fs=44100, output='sos')
    filtertrain[10] = signal.butter(n, [13000,14000], 'bandpass', fs=44100, output='sos')
    filtertrain[11] = signal.butter(n, [14000,15000], 'bandpass', fs=44100, output='sos')
    filtertrain[12] = signal.butter(n, 15000, 'highpass', fs=44100, output='sos')
    heightVar = 0.375
    distanceVar = 4000
    occurenceMatrix = [[0]*len(filtertrain)]*1
    for i in range(0,1):
        y, sr = librosa.load(song)
        for j in np.arange(len(filtertrain)):
            filtered = signal.sosfilt(filtertrain[j], y)
            ypeaks = signal.find_peaks(filtered, height=np.max(filtered)*heightVar, distance=distanceVar)
            occurenceMatrix[i][j] = len(ypeaks[0])                
    return occurenceMatrix             

In [61]:
#readInLiveAudio()
#betterVocalExtract('output.wav')
def mainProgram(song):
    occurenceMatrix = bandComparison(song)
    englishCount = 0
    frenchCount  = 0
    germanCount  = 0
    japaneseCount = 0
    dataEnglish = englishPeaks()
    dataFrench = frenchPeaks()
    dataGerman = germanPeaks()
    dataJapanese = japanesePeaks()
    for i in range(0,len(dataEnglish)):
        
        #This section is the first leg in the tree, determines if there is the most 
        #difference between highpass and lowpass filters.
        #These are the filters used as there is the greatest difference between them 
        #while summing all the filters together.
        currentVa = occurenceMatrix[0][0]
        englishVa = abs(dataEnglish[i][0] - currentVa)
        frenchVa = abs(dataFrench[i][0] - currentVa)
        germanVa = abs(dataGerman[i][0] - currentVa)
        japaneseVa = abs(dataJapanese[i][0] - currentVa)
       
        lowpass = englishVa + frenchVa + germanVa + japaneseVa
        
        currentHVA = occurenceMatrix[0][12]
        englishHVa = abs(dataEnglish[i][12]-currentHVA)
        frenchHVa = abs(dataFrench[i][12]-currentHVA)
        germanHVa = abs(dataGerman[i][12]-currentHVA)
        japaneseHVa = abs(dataJapanese[i][12]-currentHVA)
        
        highpass = englishHVa + frenchHVa + germanHVa + japaneseHVa
        
        #More of a difference in Low pass
        #The higher the difference means the more likely
        if highpass<lowpass:
            #print("Low Pass")
           
            #English and Japanese are the most similar at lower
            #ends of the spectrum, these will be explored 
            #further to find a major difference
            #The bands being explored are ones that
            #have the highest degree differnce in the dataset
            if (frenchVa>englishVa and germanVa>englishVa) or (frenchVa>japaneseVa and germanVa>japaneseVa):
                #print("English or Japanese")
                tempEnglishCount=0
                tempJapaneseCount=0          
                
                currentVa4 = occurenceMatrix[0][4]
                currentVa5 = occurenceMatrix[0][5]
                currentVa12 = occurenceMatrix[0][12]        
                
                currentE4 = abs(dataEnglish[i][4]-currentVa4)
                currentE5 = abs(dataEnglish[i][5]-currentVa5)
                currentE12 = abs(dataEnglish[i][12]-currentVa12)           
                
                currentJ4 = abs(dataJapanese[i][4]-currentVa4)
                currentJ5 = abs(dataJapanese[i][5]-currentVa5)
                currentJ12 = abs(dataJapanese[i][12]-currentVa5)
                
                #Keeps track over which values are greater than or less than
                #if uncertain doesn't add to any.
                if currentE4<currentJ4:
                    tempEnglishCount+=1
                if currentE4>currentJ4:
                    tempJapaneseCount+=1
                if currentE5<currentJ5:
                    tempEnglishCount+=1
                if currentE5>currentJ5:
                    tempJapaneseCount+=1
                if currentE12<currentJ12:
                    tempEnglishCount+=1
                if currentE12>currentJ12:
                    tempJapaneseCount+=1

                #adds 1 to the total count, to the highest tempCount
                #does nothing if they are equal
                if tempEnglishCount<tempJapaneseCount:
                    japaneseCount+=1
                   # print("Japanese")
                if tempJapaneseCount<tempEnglishCount:
                    englishCount+=1
                    #print("English")

                    
                    
            #Dataset Between German and French
            #If this dataset is reached the lowband is most 
            #likely to be German or French
            #Will explore the bands that are most 
            #different between German and French
            elif (englishVa>frenchVa and englishVa>germanVa) or (japaneseVa>frenchVa and japaneseVa>germanVa):
                    #print("Low pass German or French")
                    tempFrenchCount=0
                    tempGermanCount=0
                    
                    #5 significant bands that have a major differnce
                    #between German and French
                    currentVa8  =  occurenceMatrix[0][8]
                    currentVa9 = occurenceMatrix[0][9]
                    currentVa10= occurenceMatrix[0][10]
                    currentVa11= occurenceMatrix[0][11]
                    currentVa12= occurenceMatrix[0][12]

                    currentF8 = abs(dataFrench[i][8]-currentVa8) 
                    currentF9 = abs(dataFrench[i][9]-currentVa9)
                    currentF10 = abs(dataFrench[i][10]-currentVa10)
                    currentF11 = abs(dataFrench[i][11]-currentVa11)
                    currentF12 = abs(dataFrench[i][12]-currentVa12)

                    currentG8 = abs(dataGerman[i][8]-currentVa8)
                    currentG9  = abs(dataGerman[i][9]-currentVa9)
                    currentG10  = abs(dataGerman[i][10]-currentVa10)
                    currentG11  = abs(dataGerman[i][11]-currentVa11)
                    currentG12  = abs(dataGerman[i][12]-currentVa12)

                    #Count the amount a features closest to
                    #a particular band.
                    if currentF8<currentG8:
                        tempFrenchCount+=1
                    else:
                        tempGermanCount+=1

                    if currentF9<currentG9:
                        tempFrenchCount+=1
                    else:
                        tempGermanCount+=1

                    if currentF10<currentG10:
                        tempFrenchCount+=1    
                    else:
                        tempGermanCount+=1

                    if currentF11<currentG11:
                        tempFrenchCount+=1
                    else:
                        tempGermanCount+=1

                    if currentF12<currentG12:
                        tempFrenchCount+=1
                    else:
                        tempGermanCount+=1

                    #Add to french or german depending on
                    #the output
                    if tempGermanCount<tempFrenchCount:
                        frenchCount  += 1
                        #print("French")
                    elif tempFrenchCount<tempGermanCount:
                        germanCount += 1
                        #print("German")
            
        #More of a difference between highpass
        elif lowpass<highpass:
            print("High Pass")
            
            #The high end is far higher than the rest
            if germanHVa<japaneseHVa and germanHVa<frenchHVa and germanHVa<englishHVa:
                germanCount+=1
                print("German")
            
            #English and French are very similar
            else:     
                tempFrenchCount = 0
                tempEnglishCount = 0
                tempJapaneseCount = 0
                
                model0 = occurenceMatrix[0][0]
                model2 = occurenceMatrix[0][2]
                model5 = occurenceMatrix[0][5]
        
                eModel0 = abs(dataEnglish[i][0]-model0)
                eModel2 = abs(dataEnglish[i][2]-model2)
                eModel5 = abs(dataEnglish[i][5]-model5)
                
                fModel0 = abs(dataFrench[i][0]-model0)
                fModel2 = abs(dataFrench[i][2]-model2)
                fModel5 = abs(dataFrench[i][5]-model5)
        
                jModel0 = abs(dataJapanese[i][0]-model0)
                jModel2 = abs(dataJapanese[i][2]-model2)
                jModel5 = abs(dataJapanese[i][5]-model5)
        
        
                if eModel0 < fModel0 and eModel0<jModel0:
                    tempEnglishCount+=1
                if fModel0 < eModel0 and fModel0<jModel0:
                    tempFrenchCount+=1
                if jModel0 < eModel0 and jModel0<fModel0:
                    tempJapaneseCount+=1
                
                if eModel2 < fModel2 and eModel2<jModel2: 
                    tempEnglishCount+=1
                if fModel2 < eModel2 and fModel2<jModel2:
                    tempFrenchCount+=1
                if jModel2 < eModel2 and jModel2<fModel2:
                    tempJapaneseCount+=1
                
                
                if eModel5 < fModel5 and eModel5<jModel5:
                    tempEnglishCount+=1
                if fModel5 < eModel5 and fModel5<jModel5:
                    tempFrenchCount+=1
                if jModel5 < eModel5 and jModel5<fModel5:
                    tempJapaneseCount+=1
                    
                    
                if tempFrenchCount<tempEnglishCount and tempJapaneseCount<tempEnglishCount:
                    englishCount += 1
                    print("English")
                if tempEnglishCount<tempFrenchCount and tempJapaneseCount<tempFrenchCount:
                    frenchCount  += 1
                    print("French")
                if tempJapaneseCount > tempEnglishCount and tempJapaneseCount>tempFrenchCount:
                    japaneseCount += 1
                    print("Japanese")
    
    
    
    
    #files = glob.glob("/home/russell/SingIT/output/('output', '.wav')/*")
    #for f in files:
     #   os.remove(f)        

    languageCount= []
    languageCount.append(englishCount)
    languageCount.append(frenchCount)
    languageCount.append(germanCount)
    languageCount.append(japaneseCount)
    language=["English","French","German","Japanese"]
    indexOfLanguage=languageCount.index(max(languageCount))        
    outputLanguage = language[indexOfLanguage]
    print("The language is: " + outputLanguage)
    return outputLanguage    
        
print("Should all be English")
mainProgram("/home/russell/SingIT/outputtestEnglish/('BonJovi-YouGiveLoveABadName-KrZHPOeOxQQOut', '.wav')/vocals.wav")
#mainProgram("/home/russell/SingIT/outputtestEnglish/('BackstreetBoys-IWantItThatWay-4fndeDfaWCgOut', '.wav')/vocals.wav")
#mainProgram("/home/russell/SingIT/outputtestEnglish/('BonJovi-ItsMyLife-vx2u5uUu3DEOut', '.wav')/vocals.wav")
#mainProgram("/home/russell/SingIT/outputtestEnglish/('[OFFICIALVIDEO]Hallelujah-Pentatonix-LRP8d7hhpoQOut', '.wav')/vocals.wav")
#mainProgram("/home/russell/SingIT/outputtestEnglish/('DreamOn-3-2u2lhGO4wOut', '.wav')/vocals.wav")
#mainProgram("/home/russell/SingIT/outputtestEnglish/('GunsNRoses-SweetChildOMineOfficialMusicVideo-1w7OgIMMRc4Out', '.wav')/vocals.wav")
#mainProgram("/home/russell/SingIT/outputtestEnglish/('ItsRainingMen-4x6leDGV7gsOut', '.wav')/vocals.wav")
#mainProgram("/home/russell/SingIT/outputtestEnglish/('IWannaDancewithSomebodyWhoLovesMe-6xzN8Nt0PokOut', '.wav')/vocals.wav")
#mainProgram("/home/russell/SingIT/outputtestEnglish/('MamboNo.5aLittleBitof...-RUBvqz3ozv8Out', '.wav')/vocals.wav")

'''
print("")
print("Should all be French")
mainProgram("/home/russell/SingIT/outputtestFrench/('ALOUETTEGENTILLEALOUETTEComptinesChansonsfrançaisesFrenchsongswithEnglishsubtitles-y3-h6n-6cr0Out', '.wav')/vocals.wav")
mainProgram("/home/russell/SingIT/outputtestFrench/('Indila-DernièreDanseClipOfficiel-K5KAc5CoCukOut', '.wav')/vocals.wav")
mainProgram("/home/russell/SingIT/outputtestFrench/('Indila-TournerDansLeVide-vtNJMAyeP0sOut', '.wav')/vocals.wav")
mainProgram("/home/russell/SingIT/outputtestFrench/('LesMiserables1991ParisRevivalLaConfrontationlyrics-FRZYOFCLhS8Out', '.wav')/vocals.wav")
mainProgram("/home/russell/SingIT/outputtestFrench/('MaîtreGims-Est-cequetumaimesClipofficiel-6TpyRE_juyAOut', '.wav')/vocals.wav")
        
mainProgram("/home/russell/SingIT/outputtestFrench/('MaîtreGims-SapéscommejamaisClipofficielft.Niska-4bPGxLxogvwOut', '.wav')/vocals.wav")
mainProgram("/home/russell/SingIT/outputtestFrench/('Mika-ElleMeDitclipofficiel-NiHWwKC8WjUOut', '.wav')/vocals.wav")
mainProgram("/home/russell/SingIT/outputtestFrench/('Stromae-AlorsOnDanseClipOfficiel-VHoT4N43jK8Out', '.wav')/vocals.wav")
mainProgram("/home/russell/SingIT/outputtestFrench/('Stromae-FormidableHQ-hfOkfmPABJwOut', '.wav')/vocals.wav")
mainProgram("/home/russell/SingIT/outputtestFrench/('Stromae-TousLesMêmesClipOfficiel-CAMWdvo71lsOut', '.wav')/vocals.wav")


#print("")
#print("Should all be German")
#mainProgram("/home/russell/SingIT/outputtestGerman/('AngutenTagen-pHACKQHEVPsOut', '.wav')/vocals.wav")
#mainProgram("/home/russell/SingIT/outputtestGerman/('Heintje-Ichbaudireinschloss1968-0o7UMkXsvKoOut', '.wav')/vocals.wav")
#mainProgram("/home/russell/SingIT/outputtestGerman/('LaFeeIchbinich-bu9YVDmVni4Out', '.wav')/vocals.wav")
#mainProgram("/home/russell/SingIT/outputtestGerman/('LaFee-IchBinIch_ThisismeEnglishtranslation-XKj5iaE-6w0Out', '.wav')/vocals.wav")
#mainProgram("/home/russell/SingIT/outputtestGerman/('Nicole-EinBisschenFrieden1982-W9S3YMmIFdwOut', '.wav')/vocals.wav")

#mainProgram("/home/russell/SingIT/outputtestGerman/('ReinhardMey-ÜberdenWolkenZDF,live-lK76cnUcj8UOut', '.wav')/vocals.wav")
#mainProgram("/home/russell/SingIT/outputtestGerman/('Revolverheldfeat.MartaJandová-HaltDichanmirfest-wCcJuN47UcYOut', '.wav')/vocals.wav")
#mainProgram("/home/russell/SingIT/outputtestGerman/('SarahConnor-MeinKönig-CQHnlSbAGDcOut', '.wav')/vocals.wav")
#mainProgram("/home/russell/SingIT/outputtestGerman/('SarahConnor-Wieschöndubist-oa47ieez6n4Out', '.wav')/vocals.wav")
#mainProgram("/home/russell/SingIT/outputtestGerman/('UnteraltenJacken-jbjTsRNos5kOut', '.wav')/vocals.wav")

print("")
print("Should all be Japanese")
mainProgram("/home/russell/SingIT/outputtestJapanese/('「AMV」DAOKOxKenshiYonezu-UchiageHanabi-6Ghs7yFA538Out', '.wav')/vocals.wav")
mainProgram("/home/russell/SingIT/outputtestJapanese/('Enka-itsukihiroshi-nvvXOfLs-ngOut', '.wav')/vocals.wav")
mainProgram("/home/russell/SingIT/outputtestJapanese/('FLOW-SignMusicVideo-qpi9YXaChHIOut', '.wav')/vocals.wav")
mainProgram("/home/russell/SingIT/outputtestJapanese/('Japaneseoldsong（1972）-1xpjo8fmyy8Out', '.wav')/vocals.wav")
mainProgram("/home/russell/SingIT/outputtestJapanese/('Kana-Boon-SilhouetteOfficialMusicVideo-dlFA0Zq1k2AOut', '.wav')/vocals.wav")

mainProgram("/home/russell/SingIT/outputtestJapanese/('KieKitano→「Hazakura」-60V_ns2HukEOut', '.wav')/vocals.wav")
mainProgram("/home/russell/SingIT/outputtestJapanese/('ONEOKROCK-ClockStrikes[OfficialMusicVideo]-6YZlFdTIdzMOut', '.wav')/vocals.wav")
mainProgram("/home/russell/SingIT/outputtestJapanese/('OnePunchManSeason2Opening『AMV』-SeijakuNoApostleFull-Bd59fbFZO14Out', '.wav')/vocals.wav")
mainProgram("/home/russell/SingIT/outputtestJapanese/('打上花火_DAOKO×米津玄師Coveredbyコバソロ&春茶&伊礼亮-qBIxl_6tFfoOut', '.wav')/vocals.wav")
mainProgram("/home/russell/SingIT/outputtestJapanese/('星野源–アイデアOfficialVideo-RlUb2F-zLxwOut', '.wav')/vocals.wav")
'''



Should all be English
High Pass
High Pass
English
High Pass
High Pass
German
High Pass
English
High Pass
Japanese
High Pass
Japanese
High Pass
Japanese
High Pass
Japanese
High Pass
Japanese
High Pass
High Pass
English
High Pass
French
High Pass
English
High Pass
High Pass
High Pass
French
High Pass
High Pass
French
High Pass
French
High Pass
French
High Pass
English
High Pass
High Pass
High Pass
High Pass
High Pass
High Pass
English
High Pass
English
High Pass
High Pass
High Pass
Japanese
High Pass
High Pass
Japanese
High Pass
French
High Pass
High Pass
High Pass
Japanese
High Pass
High Pass
Japanese
High Pass
Japanese
High Pass
The language is: Japanese


'\nprint("")\nprint("Should all be French")\nmainProgram("/home/russell/SingIT/outputtestFrench/(\'ALOUETTEGENTILLEALOUETTEComptinesChansonsfrançaisesFrenchsongswithEnglishsubtitles-y3-h6n-6cr0Out\', \'.wav\')/vocals.wav")\nmainProgram("/home/russell/SingIT/outputtestFrench/(\'Indila-DernièreDanseClipOfficiel-K5KAc5CoCukOut\', \'.wav\')/vocals.wav")\nmainProgram("/home/russell/SingIT/outputtestFrench/(\'Indila-TournerDansLeVide-vtNJMAyeP0sOut\', \'.wav\')/vocals.wav")\nmainProgram("/home/russell/SingIT/outputtestFrench/(\'LesMiserables1991ParisRevivalLaConfrontationlyrics-FRZYOFCLhS8Out\', \'.wav\')/vocals.wav")\nmainProgram("/home/russell/SingIT/outputtestFrench/(\'MaîtreGims-Est-cequetumaimesClipofficiel-6TpyRE_juyAOut\', \'.wav\')/vocals.wav")\n        \nmainProgram("/home/russell/SingIT/outputtestFrench/(\'MaîtreGims-SapéscommejamaisClipofficielft.Niska-4bPGxLxogvwOut\', \'.wav\')/vocals.wav")\nmainProgram("/home/russell/SingIT/outputtestFrench/(\'Mika-ElleMeDitclipofficiel-NiHWwKC